# Zero Shot: JMS -> SmallRye Reactive - jms-to-reactive-quarkus-00010

> Work with a LLM to generate a fix for the rule: jms-to-reactive-quarkus-00010

This is a simpler example that will ignore the solved example and only use source code analysis information

##### Sample Applications Used
* 2 sample apps from [JBoss EAP Quickstarts](https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x) were chosen:  helloworld-mdb & cmt
    * [helloworld-mdb](https://github.com/savitharaghunathan/helloworld-mdb)
    * [cmt](https://github.com/konveyor-ecosystem/cmt)

##### Using Custom Rules for JMS to SmallRye Reactive
* Rules were developed by Juanma [@jmle](https://github.com/jmle)
    * Rules originally from: https://github.com/jmle/rulesets/blob/jms-rule/default/generated/quarkus/05-jms-to-reactive-quarkus.windup.yaml


In [1]:
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

Obtaining file:///Users/jmatthews/git/jwmatthews/kai
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kai
    Found existing installation: kai 0.0.1
    Uninstalling kai-0.0.1:
      Successfully uninstalled kai-0.0.1
  Running setup.py develop for kai


In [2]:
# Bring in the common deps
import os
import pprint

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

pp = pprint.PrettyPrinter(indent=2)


def get_java_in_result(text: str):
    _, after = text.split("```java")
    return after.split("```")[0]


def write_output_to_disk(
    out_dir,
    formatted_template,
    example_javaee_file_contents,
    example_quarkus_file_contents,
    result,
):
    try:
        os.makedirs(out_dir, exist_ok=True)
    except OSError as error:
        print(f"Error creating directory {out_dir}: {error}")
        raise error

    # Save the template to disk
    template_path = os.path.join(out_dir, "template.txt")
    with open(template_path, "w") as f:
        f.truncate()
        f.write(formatted_template)
    print(f"Saved template to {template_path}")

    # Save the result
    result_path = os.path.join(out_dir, "result.txt")
    with open(result_path, "w") as f:
        f.truncate()
        f.write(result)
    print(f"Saved result to {result_path}")

    # Extract the updated java code and save it
    updated_file_contents = get_java_in_result(result)
    updated_file_path = os.path.join(out_dir, "updated_file.java")
    with open(updated_file_path, "w") as f:
        f.truncate()
        f.write(updated_file_contents)
    print(f"Saved updated java file to {updated_file_path}")

    # Save the original source code
    original_file_path = os.path.join(out_dir, "original_file.java")
    with open(original_file_path, "w") as f:
        f.truncate()
        f.write(example_javaee_file_contents)
    print(f"Saved original java file to {original_file_path}")

    # Save the Quarkus version we already in Git to use as a comparison
    known_quarkus_file_path = os.path.join(out_dir, "quarkus_version_from_git.java")
    with open(known_quarkus_file_path, "w") as f:
        f.truncate()
        f.write(example_quarkus_file_contents)
    print(f"Saved Quarkus version from Git to {known_quarkus_file_path}")

# Create a Prompt
## Step #1:  Create a Prompt Template


In [3]:
template_with_analysis_only = """
    # Java EE to Quarkus Migration
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.
    Pay attention to changes you make and impacts to external dependencies in the pom.xml.  
    As you make changes that impact the pom.xml be sure you explain what needs to be updated in pom.xml.
    
    In addition, I will provide you with a list of issues that were found in the JavaEE code which
    we believe need to be addressed before being migrated to Quarkus.  Be sure to address these 
    issues or explain why you do not think the issue is relevant.

    Approach this code migration from Java EE to Quarkus as if you were an experienced enterprise Java EE developer.
    Before attempting to migrate the code to Quarkus, explain each step of your reasoning through what changes 
    are required and why. 

    After you have shared your step by step thinking, provide a full output of the updated file:

    # Input information
    ## Issue found from static code analysis of the Java EE code which needs to be fixed to migrate to Quarkus
    Issue to fix:  "{analysis_message}"

    ## Line number of the issue first appearing in the Java EE code source code example below
    Line number: {analysis_lineNumber}

    ## Input file name
    Filename: "{src_file_name}"

    ## Input source code file contents for "{src_file_name}"
    ```java 
    {src_file_contents}
    ```

    # Output Instructions
    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

# Step 2: Gather the data we want to inject into the prompt

In [4]:
from kai.report import Report
from kai.scm import GitDiff

# Static code analysis was run prior and committed to this repo
path_helloworld_analysis = "./analysis_report/helloworld-mdb-quarkus/output.yaml"

helloworld_report = Report.load_report_from_file(path_helloworld_analysis)

# We are limiting our experiment to a single rule for this run
ruleset_name = "custom-ruleset"
rule = "jms-to-reactive-quarkus-00010"
helloworld_rule_data = helloworld_report.report[ruleset_name]["violations"][rule]

# We are expecting to find 2 impacted files in HelloWorld
assert len(helloworld_rule_data["incidents"]), 2

# Setup access to look at the Git repo source code for each sample app
helloworld_src_path = "../../kai_solution_server/samples/sample_repos/helloworld-mdb-quarkus"
helloworld_diff = GitDiff(helloworld_src_path)

# We want to be sure the the 'quarkus' branch of both repos has been checked out
# so it's available to consult
helloworld_diff.checkout_branch("quarkus")
# Resetting to 'main' as what is currently checked out
helloworld_diff.checkout_branch("main")

# Now we can extract the info we need
## We will assume:
## .   HelloWorld will be our source application to migrate, so we will use it's 'main' branch which maps to Java EE

hw_entry = helloworld_rule_data["incidents"][0]
src_file_name = helloworld_report.get_cleaned_file_path(hw_entry["uri"])
src_file_contents = helloworld_diff.get_file_contents(src_file_name, "main")
analysis_message = hw_entry["message"]
analysis_lineNumber = hw_entry["lineNumber"]

Reading report from ./analysis_report/helloworld-mdb-quarkus/output.yaml


# Step 3: Run the prompt through the LLM and write the output to disk

In [5]:
# For first run we will only include the source file and analysis info (we are omitting solved example)
template_args = {
    "src_file_name": src_file_name,
    "src_file_contents": src_file_contents,
    "analysis_message": analysis_message,
    "analysis_lineNumber": analysis_lineNumber,
}
formatted_prompt = template_with_analysis_only.format(**template_args)

model_name = "gpt-4-1106-preview"
# model_name="gpt-3.5-turbo-16k"

llm = ChatOpenAI(temperature=0.1, model_name=model_name)
prompt = PromptTemplate.from_template(template_with_analysis_only)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(template_args)

# For comparison, we will look up what the source file looks like from Quarkus branch
# this serves as a way of comparing to what the 'answer' is that was done manually by a knowledgeable developer
src_file_from_quarkus_branch = helloworld_diff.get_file_contents(
    src_file_name, "quarkus"
)

output_dir = (
    f"output/{model_name}/helloworldmdb/{ruleset_name}/{rule}/zero_shot_with_analysis/"
)
write_output_to_disk(
    output_dir,
    formatted_prompt,
    src_file_contents,
    src_file_from_quarkus_branch,
    result,
)

Saved template to output/gpt-4-1106-preview/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/zero_shot_with_analysis/template.txt
Saved result to output/gpt-4-1106-preview/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/zero_shot_with_analysis/result.txt
Saved updated java file to output/gpt-4-1106-preview/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/zero_shot_with_analysis/updated_file.java
Saved original java file to output/gpt-4-1106-preview/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/zero_shot_with_analysis/original_file.java
Saved Quarkus version from Git to output/gpt-4-1106-preview/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00010/zero_shot_with_analysis/quarkus_version_from_git.java
